In [1]:
logging = False



def my_euler_maclaurin1(f,symb,a,b,p,calc_R_p=False):
    if logging:
        print("f", f)
        print("symb,a,b", symb, a, b)
        print("")
    u, j = var("u, j")

    sum_dfdx_a_bernoullis = sum([(f.diff(symb,k-1))*(bernoulli(k)/factorial(k)) for k in range(1,1+p)])
    if logging:
        print("sum_dfdx_a_bernoullis", sum_dfdx_a_bernoullis)
        print("sum_dfdx_a_bernoullis(a)", sum_dfdx_a_bernoullis.subs(symb == a))
        if Infinity != b:
            print("sum_dfdx_a_bernoullis(b)", sum_dfdx_a_bernoullis.subs(symb == b))

    if True:
        B = lambda x, p : bernoulli_polynomial(x, p)
        fact = lambda n : factorial(n)
    else:
        B = function("B")(x,p)
        fact = function("fact")(n)
        
    if True == calc_R_p:
        # Can't calculate derivative wrt j + u.
        #int_R_p = lambda j : integral(f.diff(symb,p).subs(symb == u+j) * bernoulli_polynomial(u,p)/factorial(p), (u,0,1))
        def int_R_p_(j):
            deriv = f.diff(symb,p)
            if logging:
                print("deriv", deriv)
            deriv_s = deriv.subs(symb == u+j)
            if logging:
                print("deriv_s", deriv_s)
            int_f_diff_B_fact = integral(deriv_s * bernoulli_polynomial(u,p)/factorial(p), (u,0,1))
            if logging:
                print("int_f_diff_B_fact", int_f_diff_B_fact)
            return int_f_diff_B_fact
        
        def int_R_p(f,symb,j):
            u = var("u")
            fn_shifted = f.subs(symb == u + j)
            if logging:
                print("fn_shifted", fn_shifted)
            deriv = fn_shifted.diff(u,p)
            if logging:
                print("deriv", deriv)
            int_f_diff_B_fact = integral(deriv * B(x=u,p=p)/fact(n=p), (u,0,1))
    
            if logging:
                print("int_f_diff_B_fact", int_f_diff_B_fact)

            return int_f_diff_B_fact

        #print("int_R_p(j)", int_R_p(j))
        if logging:
            print("int_R_p(j = a)", int_R_p(f,symb,j = a))
        assume(j>0)
        R_p = (-1)^(p+1)*sum(int_R_p(f,symb,j=j), j, a, b-1)

    int_fn_a_b = integral(f, (symb, a, b))
    if logging:
        print("int_fn_a_b", int_fn_a_b)
    
    #s = int_fn_a_b + sum_dfdx_a_bernoullis(b) - sum_dfdx_a_bernoullis(a) + R_p
    s = int_fn_a_b - sum_dfdx_a_bernoullis.subs(symb == a)
    if Infinity != b:
        s += sum_dfdx_a_bernoullis.subs(symb == b)
    if True == calc_R_p:
        s += R_p
        
    if False:
        print("int_fn_a_b", int_fn_a_b)
        print("sum_dfdx_a_bernoullis(a)", sum_dfdx_a_bernoullis(a))
        if Infinity != b:
            print("sum_dfdx_a_bernoullis(b)", sum_dfdx_a_bernoullis(b))
        if True == calc_R_p:
            print("int_R_p(j = a)", int_R_p(j = a))
            print("R_p", R_p)
        print("")
        print("s",s)
    return s

$${\displaystyle \sum _{i=a}^{b}f(i)=\int _{a}^{b}f(x)\,dx+{\frac {f(a)+f(b)}{2}}+\sum _{k=1}^{\lfloor p/2\rfloor }{\frac {B_{2k}}{(2k)!}}(f^{(2k-1)}(b)-f^{(2k-1)}(a))+R_{p},}$$

$${\displaystyle \sum _{i=a}^{b}f(i)=\int _{a}^{b}f(x)\,dx+\sum _{k=1}^{\lfloor p/2\rfloor }{\frac {B_{k}}{(k)!}}(f^{(k-1)}(b)-f^{(k-1)}(a))+R_{p},}$$

$${\displaystyle R_{p}=(-1)^{p+1}\sum_{j=a}^{b-1} \int _{0}^{1}f^{(p)}(u+j){\frac {B_{p}(u)}{p!}}\,du.}$$

In [2]:
if True:
    B = lambda x, p : bernoulli_polynomial(x, p)
    fact = lambda n : factorial(n)
else:
    B = function("B")(x,p)
    fact = function("fact")(n)

def euler_maclaurin_R_p(f,symb,a,b,p):
    
    def int_R_p_(f,symb,jx):
        ux = var("u_"+str(symb))
        deriv = f.diff(symb,p)
        if logging:
            print("deriv", deriv)
            
        deriv_subs = deriv.subs(symb == ux+jx)
        if logging:
            print("deriv_subs", deriv_subs)

        int_f_diff_B_fact = integral(deriv_subs * bernoulli_polynomial(u,p)/factorial(p), (u,0,1))
        if logging:
            print("int_f_diff_B_fact", int_f_diff_B_fact)
            
        return int_f_diff_B_fact
    
    def int_R_p(f,symb,jx):
        ux = var("u_"+str(symb))
        fn_shifted = f.subs(symb == ux + jx)
        if logging:
            print("fn_shifted", fn_shifted)

        deriv = fn_shifted.diff(ux,p)
        if logging:
            print("deriv", deriv)

        int_f_diff_B_fact = integral(deriv * B(x=ux,p=p)/fact(n=p), (ux, 0, 1))

        if logging:
            print("int_f_diff_B_fact", int_f_diff_B_fact)

        return int_f_diff_B_fact
    
    jx = var('j_'+str(symb))
    assume(jx>0)
    if logging:
        print("f", f)
        print("symb,a,b,jx", symb, a, b, jx)
        print("")
    R_p = (-1)^(p+1)*sum(int_R_p(f=f, symb=symb, jx=jx), jx, a, b-1)
    return R_p


def sum_dfdx_bernoulis(f,symb,a,b,p):
    dfdx_a_bernoullis = []
    for k in range(1,1+p):
        dfdx_a_bernoullis += [(f.diff(symb,k-1))*(bernoulli(k)/factorial(k))]
        
    sum_dfdx_a_bernoullis = sum(dfdx_a_bernoullis)

    if logging:
        print("sum_dfdx_a_bernoullis", sum_dfdx_a_bernoullis)
        print("sum_dfdx_a_bernoullis(a)", sum_dfdx_a_bernoullis.subs(symb == a))
        if Infinity != b:
            print("sum_dfdx_a_bernoullis(b)", sum_dfdx_a_bernoullis.subs(symb == b))

    s = - sum_dfdx_a_bernoullis.subs(symb == a)
    if Infinity != b:
        s += sum_dfdx_a_bernoullis.subs(symb == b)
    return s

def definite_integral_derivative(f, x, a, b, diff_param=None):
    from sage.symbolic.integration.integral import definite_integral

    if not x.has(diff_param):
        # integration variable != differentiation variable
        ans = definite_integral(f.diff(diff_param), x, a, b, hold=True)
    else:
        ans = SR.zero()

    if hasattr(b, 'diff'):
        if b.diff(diff_param) != 0:
            ans += f.subs(x == b) * b.diff(diff_param)
    if hasattr(a, 'diff'):
        if a.diff(diff_param) != 0:
            ans -= f.subs(x == a) * a.diff(diff_param)

    return ans

#sum_R_p = sum_dfdx_bernoulis(euler_maclaurin_R_p(F(x,y), x, a_x, b_x,p), y, a_y, b_y, p)
def sum_dfdx_bernoulis_euler_maclaurin_R_p(f, x, a_x, b_x, y, a_y, b_y, p):
    dfdx_a_bernoullis = []
    from sage.calculus.calculus import symbolic_sum
    ux, jx = var("u_x, j_x")
    for k in range(1,1+p):
        if 1 == k:
            term_1 = lambda y, jx : f(x,y).diff(x,p).subs(x == ux + jx) * B(x=ux,p=p)/fact(n=p)
            if logging:
                print("term_1(a_x) =", term_1(y=y,jx=a_x))
            R_p = lambda y : (-1)^(p+1)*symbolic_sum(expression=term_1(y=y,jx=jx), v=jx, a=a_x, b=b_x-1)            
            if logging:
                print("R_p =", R_p(y))
            dfdx_a_bernoullis += [lambda y, i=i : R_p(y)]
        elif 2 == k:
            term_2 = lambda y, jx : definite_integral_derivative(f(x,y).diff(x,p).subs(x == ux + jx) \
                                                              * B(x=ux,p=p)/fact(n=p), ux, 0, 1, y)
            
            if logging:
                print("term_2(a_x) =", term_2(y=y,jx=a_x))
                
            R_p = lambda y : (-1)^(p+1)*symbolic_sum(expression=term_2(y=y,jx=jx), v=jx, a=a_x, b=b_x-1)            
            if logging:
                print("R_p =", R_p(y))
            dfdx_a_bernoullis += [lambda y, i=i : R_p(y)]
        else:
            term_3 = lambda y,jx : definite_integral_derivative(f(x,y).diff(x,p).subs(x == ux + jx) \
                                                              * B(x=ux,p=p)/fact(n=p), ux, 0, 1, x).diff(y, k-2)
            
            if logging:
                print("term_3(a_x) =", term_3(y=x,jx=a_x))
            R_p = lambda y : (-1)^(p+1)*symbolic_sum(expression=term_2(y=y,jx=jx), v=jx, a=a_x, b=b_x-1)            
            if logging:
                print("R_p =", R_p(y))
            dfdx_a_bernoullis += [lambda y, i=i : R_p(y)]

    if logging:
        print("dfdx_a_bernoullis =", dfdx_a_bernoullis)
        print("len(dfdx_a_bernoullis) =", len(dfdx_a_bernoullis))

    sum_dfdx_a_bernoullis = lambda y : sum([dfdx_a_bernoullis[i](y) for i in range(len(dfdx_a_bernoullis))])
    
    if logging:
        print("sum_dfdx_a_bernoullis =", sum_dfdx_a_bernoullis)
        print("sum_dfdx_a_bernoullis(a_y) =", sum_dfdx_a_bernoullis(y=a_y))
    
    s = - sum_dfdx_a_bernoullis(a_y)
    if Infinity != b_y:
        s += sum_dfdx_a_bernoullis(b_y)
    return s

#R_p_R_p = euler_maclaurin_R_p(euler_maclaurin_R_p(F(x,y), x, a_x, b_x,p), y, a_y, b_y, p)
def euler_maclaurin_R_p_euler_maclaurin_R_p(f, x, a_x, b_x, y, a_y, b_y, p):

    from sage.calculus.calculus import symbolic_sum
    from sage.symbolic.integration.integral import definite_integral
    ux,uy = var("u_x, u_y")
    jx,jy = var("j_x, j_y")

    R_p_R_p = (-1)^(p+1)*symbolic_sum(definite_integral( \
                           (-1)^(p+1)*symbolic_sum(definite_integral(f.diff(x,p).subs(x == ux + jx)  \
                                                          * B(x=ux,p=p)/fact(n=p), \
                             ux,0,1,hold=True), v=jx, a=a_x, b=b_x-1,hold=True)  .diff(y,p).subs(y == uy + jy)  \
                                                          * B(x=uy,p=p)/fact(n=p), \
                             uy,0,1,hold=True), v=jy, a=a_y, b=b_y-1,hold=True)
    return R_p_R_p



def euler_maclaurin1(f,symb,a,b,p,calc_R_p=False):
    if logging:
        print("f", f)
        print("symb,a,b", symb, a, b)
        print("")
    #u, j = var("u, j")

    int_fn_a_b = integral(f, (symb, a, b))
    if logging:
        print("int_fn_a_b", int_fn_a_b)
    
    #s = int_fn_a_b + sum_dfdx_a_bernoullis(b) - sum_dfdx_a_bernoullis(a) + R_p
    s = int_fn_a_b + sum_dfdx_bernoulis(f,symb,a,b,p)
    
    if True == calc_R_p:
        s += euler_maclaurin_R_p(f,symb,a,b,p)

    return s

In [3]:
from IPython.display import display, Math, Latex

In [4]:
a_x = var("n_x")
b_x = var("b_x")
a_y = var("n_y")
b_y = var("b_y")

assume(a_x, 'real')
assume(b_x, 'real')
assume(a_y, 'real')
assume(b_y, 'real')

assume(a_x, 'integer')
assume(b_x, 'integer')
assume(a_y, 'integer')
assume(b_y, 'integer')

assume(a_x > 0)
assume(b_x > 0)
assume(a_y > 0)
assume(b_y > 0)

x,y = var("x,y")

n = QQ['n'].0
j = QQ['j'].0

p = 4
p = 2

F = function("F")(x, y)
print(F)

F(x, y)


In [5]:
my_euler_maclaurin1(F(x,y),x,a_x,b_x,p, calc_R_p=True)

1/2*F(n_x, y) + integrate(F(x, y), x, n_x, b_x) - 1/12*sum(integrate(6*u^2*D[0, 0](F)(j + u, y) - 6*u*D[0, 0](F)(j + u, y) + D[0, 0](F)(j + u, y), u, 0, 1), j, n_x, b_x - 1) - 1/12*diff(F(n_x, y), n_x)

In [6]:
ans = euler_maclaurin1(F(x,y),x,a_x,b_x,p, calc_R_p=True)

In [7]:
display(Math(latex(ans)))

<IPython.core.display.Math object>

In [8]:
my_euler_maclaurin1(F(x,y),x,a_x,b_x,p)

1/2*F(n_x, y) + integrate(F(x, y), x, n_x, b_x) - 1/12*diff(F(n_x, y), n_x)

In [9]:
euler_maclaurin1(F(x,y),x,a_x,b_x,p)

1/2*F(n_x, y) + integrate(F(x, y), x, n_x, b_x) - 1/12*diff(F(n_x, y), n_x)

In [10]:
_int = integral           (F(x,y),(x, a_x, b_x))
_sum = sum_dfdx_bernoulis (F(x,y), x, a_x, b_x,p)
_R_p = euler_maclaurin_R_p(F(x,y), x, a_x, b_x,p)

display(Math(latex(_int)))
display(Math(latex(_sum)))
display(Math(latex(_R_p)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

$${\displaystyle R_{p}=(-1)^{p+1}\sum_{j=a}^{b-1} \int _{0}^{1}f^{(p)}(u+j){\frac {B_{p}(u)}{p!}}\,du.}$$

In [11]:
# sxy2 = my_euler_maclaurin1(my_euler_maclaurin1(f(x,y),x,a_x,b_x,p, calc_R_p=True),y,a_y,b_y,p, calc_R_p=True)
# Can't calculate derivative wrt j + u.

# integral(f, (symb, a, b)) + sum_dfdx_bernoulis(f,symb,a,b,p) + euler_maclaurin_R_p(f,symb,a,b,p)

int_int = integral(integral           (F(x,y),(x, a_x, b_x)),  (y, a_y, b_y)) 
int_sum = integral(sum_dfdx_bernoulis (F(x,y), x, a_x, b_x,p), (y, a_y, b_y)) 
int_R_p = integral(euler_maclaurin_R_p(F(x,y), x, a_x, b_x,p), (y, a_y, b_y))

print("int_int=")
display(Math(latex(int_int)))
print("int_sum=")
display(Math(latex(int_sum)))
print("int_R_p=")
display(Math(latex(int_R_p)))

int_int=


<IPython.core.display.Math object>

int_sum=


<IPython.core.display.Math object>

int_R_p=


<IPython.core.display.Math object>

In [12]:
# sxy2 = my_euler_maclaurin1(my_euler_maclaurin1(f(x,y),x,a_x,b_x,p, calc_R_p=True),y,a_y,b_y,p, calc_R_p=True)
# Can't calculate derivative wrt j + u.

# integral(f, (symb, a, b)) + sum_dfdx_bernoulis(f,symb,a,b,p) + euler_maclaurin_R_p(f,symb,a,b,p)

sum_int = sum_dfdx_bernoulis(integral           (F(x,y),(x, a_x, b_x)),  y, a_y, b_y, p)
sum_sum = sum_dfdx_bernoulis(sum_dfdx_bernoulis (F(x,y), x, a_x, b_x,p), y, a_y, b_y, p)
#sum_R_p = sum_dfdx_bernoulis(euler_maclaurin_R_p(F(x,y), x, a_x, b_x,p), y, a_y, b_y, p)
sum_R_p = sum_dfdx_bernoulis_euler_maclaurin_R_p(F(x,y), x, a_x, b_x,    y, a_y, b_y, p)

print("sum_int=")
display(Math(latex(sum_int)))
print("sum_sum=")
display(Math(latex(sum_sum)))
print("sum_R_p=")
display(Math(latex(sum_R_p)))

<ipython-input-2-9ea59e39fb34>:109: DeprecationWarning: Substitution using function-call syntax and unnamed arguments is deprecated and will be removed from a future release of Sage; you can use named arguments instead, like EXPR(x=..., y=...)
See http://trac.sagemath.org/5930 for details.
  R_p = lambda y : (-Integer(1))**(p+Integer(1))*symbolic_sum(expression=term_2(y=y,jx=jx), v=jx, a=a_x, b=b_x-Integer(1))


sum_int=


<IPython.core.display.Math object>

sum_sum=


<IPython.core.display.Math object>

sum_R_p=


<IPython.core.display.Math object>

In [13]:
# sxy2 = my_euler_maclaurin1(my_euler_maclaurin1(f(x,y),x,a_x,b_x,p, calc_R_p=True),y,a_y,b_y,p, calc_R_p=True)
# Can't calculate derivative wrt j + u.

# integral(f, (symb, a, b)) + sum_dfdx_bernoulis(f,symb,a,b,p) + euler_maclaurin_R_p(f,symb,a,b,p)

R_p_int = euler_maclaurin_R_p(integral           (F(x,y),(x, a_x, b_x)),  y, a_y, b_y, p)
R_p_sum = euler_maclaurin_R_p(sum_dfdx_bernoulis (F(x,y), x, a_x, b_x,p), y, a_y, b_y, p)
#R_p_R_p = euler_maclaurin_R_p(euler_maclaurin_R_p(F(x,y), x, a_x, b_x,p), y, a_y, b_y, p)
R_p_R_p = euler_maclaurin_R_p_euler_maclaurin_R_p(F(x,y), x, a_x, b_x,    y, a_y, b_y, p)

print("R_p_int=")
display(Math(latex(R_p_int)))
print("R_p_sum=")
display(Math(latex(R_p_sum)))
print("R_p_R_p=")
display(Math(latex(R_p_R_p)))

R_p_int=


<IPython.core.display.Math object>

R_p_sum=


<IPython.core.display.Math object>

R_p_R_p=


<IPython.core.display.Math object>

In [14]:
sxy_symb = euler_maclaurin1(euler_maclaurin1(F(x,y),x,a_x,b_x,p),y,a_y,b_y,p)

In [15]:
sxy_symb

1/4*F(n_x, n_y) - 1/12*integrate(-6*F(n_x, y), y, n_y, b_y) - 1/12*integrate(-12*integrate(F(x, y), x, n_x, b_x), y, n_y, b_y) + 1/2*integrate(F(x, n_y), x, n_x, b_x) - 1/12*integrate(diff(F(n_x, y), n_x), y, n_y, b_y) - 1/12*integrate(diff(F(x, n_y), n_y), x, n_x, b_x) - 1/24*diff(F(n_x, n_y), n_x) + 1/144*diff(F(n_x, n_y), n_x, n_y) - 1/24*diff(F(n_x, n_y), n_y)

In [16]:
latex(sxy_symb)

\frac{1}{4} \, F\left(n_{x}, n_{y}\right) - \frac{1}{12} \, \int_{n_{y}}^{b_{y}} -6 \, F\left(n_{x}, y\right)\,{d y} - \frac{1}{12} \, \int_{n_{y}}^{b_{y}} -12 \, \int_{n_{x}}^{b_{x}} F\left(x, y\right)\,{d x}\,{d y} + \frac{1}{2} \, \int_{n_{x}}^{b_{x}} F\left(x, n_{y}\right)\,{d x} - \frac{1}{12} \, \int_{n_{y}}^{b_{y}} \frac{\partial}{\partial n_{x}}F\left(n_{x}, y\right)\,{d y} - \frac{1}{12} \, \int_{n_{x}}^{b_{x}} \frac{\partial}{\partial n_{y}}F\left(x, n_{y}\right)\,{d x} - \frac{1}{24} \, \frac{\partial}{\partial n_{x}}F\left(n_{x}, n_{y}\right) + \frac{1}{144} \, \frac{\partial^{2}}{\partial n_{x}\partial n_{y}}F\left(n_{x}, n_{y}\right) - \frac{1}{24} \, \frac{\partial}{\partial n_{y}}F\left(n_{x}, n_{y}\right)

In [17]:
display(Math(latex(sxy_symb)))

<IPython.core.display.Math object>

In [18]:
f(x,y)=1/(x^2)
assume(y>0)
assume(x>0)
a_x = 1
b_x = Infinity

a_y = 1
b_y = Infinity

p = 4

In [19]:
my_euler_maclaurin1(f, x, a_x, b_x, p, calc_R_p=True)

(x, y) |--> 1/6*pi^2

In [20]:
euler_maclaurin1(f, x, a_x, b_x, p, calc_R_p=True)

(x, y) |--> 1/6*pi^2

In [21]:
my_euler_maclaurin1(f,x,a_x,b_x,p)

(x, y) |--> 49/30

In [22]:
euler_maclaurin1(f,x,a_x,b_x,p)

(x, y) |--> 49/30

In [23]:
f(x,y)=1/(x^2+y^2)
assume(y>0)
assume(x>0)

In [24]:
euler_maclaurin1(f,x,a_x,b_x,p)

(x, y) |--> 1/2*pi/y - arctan(1/y)/y + 1/2/(y^2 + 1) + 1/6/(y^2 + 1)^2 + 1/30/(y^2 + 1)^3 - 1/15/(y^2 + 1)^4

In [25]:
euler_maclaurin1(f,y,a_y,b_y,p)

(x, y) |--> 1/2*pi/x - arctan(1/x)/x + 1/2/(x^2 + 1) + 1/6/(x^2 + 1)^2 + 1/30/(x^2 + 1)^3 - 1/15/(x^2 + 1)^4

In [26]:
my_euler_maclaurin1(f,x,a_x,b_x,p, calc_R_p=True)

(x, y) |--> 1/2*pi/y - arctan(1/y)/y + 1/2/(y^2 + 1) + 1/6/(y^2 + 1)^2 + 1/30/(y^2 + 1)^3 - 1/15/(y^2 + 1)^4 - sum(1/30*(30*y^16*(arctan((j + 1)/y) - arctan(j/y)) + 120*((2*j^2 + 2*j + 1)*arctan((j + 1)/y) - (2*j^2 + 2*j + 1)*arctan(j/y))*y^14 - 30*y^15 - 10*(21*j^2 + 21*j + 11)*y^13 + 60*((14*j^4 + 28*j^3 + 26*j^2 + 12*j + 3)*arctan((j + 1)/y) - (14*j^4 + 28*j^3 + 26*j^2 + 12*j + 3)*arctan(j/y))*y^12 - 2*(315*j^4 + 630*j^3 + 590*j^2 + 275*j + 73)*y^11 + 120*((14*j^6 + 42*j^5 + 57*j^4 + 44*j^3 + 21*j^2 + 6*j + 1)*arctan((j + 1)/y) - (14*j^6 + 42*j^5 + 57*j^4 + 44*j^3 + 21*j^2 + 6*j + 1)*arctan(j/y))*y^10 - (1050*j^6 + 3150*j^5 + 4250*j^4 + 3250*j^3 + 1538*j^2 + 438*j + 79)*y^9 + 30*((70*j^8 + 280*j^7 + 500*j^6 + 520*j^5 + 346*j^4 + 152*j^3 + 44*j^2 + 8*j + 1)*arctan((j + 1)/y) - (70*j^8 + 280*j^7 + 500*j^6 + 520*j^5 + 346*j^4 + 152*j^3 + 44*j^2 + 8*j + 1)*arctan(j/y))*y^8 - (1050*j^8 + 4200*j^7 + 7400*j^6 + 7500*j^5 + 4788*j^4 + 1976*j^3 + 532*j^2 + 94*j + 15)*y^7 + 120*((14*j^10 + 70*

In [27]:
my_euler_maclaurin1(f,y,a_y,b_y,p, calc_R_p=True)

(x, y) |--> 1/2*pi/x - arctan(1/x)/x + 1/2/(x^2 + 1) + 1/6/(x^2 + 1)^2 + 1/30/(x^2 + 1)^3 - 1/15/(x^2 + 1)^4 - sum(1/30*(30*x^16*(arctan((j + 1)/x) - arctan(j/x)) + 120*((2*j^2 + 2*j + 1)*arctan((j + 1)/x) - (2*j^2 + 2*j + 1)*arctan(j/x))*x^14 - 30*x^15 - 10*(21*j^2 + 21*j + 11)*x^13 + 60*((14*j^4 + 28*j^3 + 26*j^2 + 12*j + 3)*arctan((j + 1)/x) - (14*j^4 + 28*j^3 + 26*j^2 + 12*j + 3)*arctan(j/x))*x^12 - 2*(315*j^4 + 630*j^3 + 590*j^2 + 275*j + 73)*x^11 + 120*((14*j^6 + 42*j^5 + 57*j^4 + 44*j^3 + 21*j^2 + 6*j + 1)*arctan((j + 1)/x) - (14*j^6 + 42*j^5 + 57*j^4 + 44*j^3 + 21*j^2 + 6*j + 1)*arctan(j/x))*x^10 - (1050*j^6 + 3150*j^5 + 4250*j^4 + 3250*j^3 + 1538*j^2 + 438*j + 79)*x^9 + 30*((70*j^8 + 280*j^7 + 500*j^6 + 520*j^5 + 346*j^4 + 152*j^3 + 44*j^2 + 8*j + 1)*arctan((j + 1)/x) - (70*j^8 + 280*j^7 + 500*j^6 + 520*j^5 + 346*j^4 + 152*j^3 + 44*j^2 + 8*j + 1)*arctan(j/x))*x^8 - (1050*j^8 + 4200*j^7 + 7400*j^6 + 7500*j^5 + 4788*j^4 + 1976*j^3 + 532*j^2 + 94*j + 15)*x^7 + 120*((14*j^10 + 70*

In [28]:
my_sxy1 = my_euler_maclaurin1(my_euler_maclaurin1(f,x,a_x,b_x,p),y,a_y,b_y,p)

In [29]:
sxy1 = euler_maclaurin1(euler_maclaurin1(f,x,a_x,b_x,p),y,a_y,b_y,p)

In [30]:
display(Math(latex(my_sxy1)))

<IPython.core.display.Math object>

In [31]:
display(Math(latex(sxy1)))

<IPython.core.display.Math object>

In [32]:
my_sxy1.n()

103.683858824338

In [33]:
sxy1.n()

103.683858824338

In [34]:
#sxy2 = my_euler_maclaurin1(my_euler_maclaurin1(f,x,a_x,b_x,p, calc_R_p=True),y,a_y,b_y,p, calc_R_p=True)

In [35]:
#sxy2 = my_euler_maclaurin1(my_euler_maclaurin1(f,x,a_x,b_x,p),y,a_y,b_y,p, calc_R_p=True)

In [36]:
#sxy2

In [37]:
#display(Math(latex(sxy2)))

In [38]:
display(Math(latex(sum(1/(x^2+y^2),x,1,Infinity))))

<IPython.core.display.Math object>

In [39]:
display(Math(latex(sum(sum(1/(x^2+y^2),x,1,Infinity),y,1,Infinity))))

<IPython.core.display.Math object>